In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.insert(0,'..')
import folium
import branca
import crossfolium

In [2]:
N = 100

data = pd.DataFrame(index=range(N))
data['letter'] = list(map(
    lambda x: 'A' if x < 0.2 else 'B' if x<0.6 else 'C',
    np.random.uniform(size=N)
    ))
data['name'] =  list(map(
    lambda x: 'Alpha' if x < 0.2 else 'Beta' if x<0.6 else 'Gamma',
    np.random.uniform(size=N)
    ))
data['lat'] = 35. + 20*np.random.uniform(size=N)
data['lng'] = -10.+ 40*np.random.uniform(size=N)

data['icon'] = data.letter.apply(lambda x: 'question' if x=='A' else 'play' if x=='B' else 'pause')
data['color'] = data.letter.apply(lambda x: 'red' if x=='A' else 'green' if x=='B' else 'blue')

In [3]:
f = branca.element.Figure(height=1000)
c = crossfolium.Crossfilter(
    data.to_dict(orient='records'),
    width='96%',
    left='2%',
    height='96%',
    top='2%',
    ).add_to(f)

c.add_subplot(2,2,1, margin=0.01).add_child(
    folium.Map([45,10], zoom_start=4, tiles='cartodbpositron').add_child(
        crossfolium.MarkerClusterFilter(
            c,
            name='mkf',
            fit_bounds=False,
            max_cluster_radius=None,
            geofilter=True,
            circle_radius=None,
            color='#0000ff',
            opacity=1.0,
            )))

c.add_subplot(4,4,3, margin=0.01).add_child(crossfolium.PieFilter(
    c,
    'letter',
    name='letter',
    order=['A','B','C'],
    colors=['#ff0000','#00ff00', '#0000ff'],
    ))

c.add_subplot(4,4,4, margin=0.01).add_child(crossfolium.PieFilter(
    c,
    'name',
    name='name',
    order=['Alpha', 'Beta', 'Gamma'],
    colors=['#00ffff','#ff00ff', '#ffff00'],
    ))

c.add_subplot(8,2,6, margin=0.01).add_child(
    crossfolium.BarFilter(
        c,
        'lat', width=400, height=100,
        bar_padding=0.1, domain=[35,55],
        groupby=1,
        elastic_y=True,
        xlabel='',
        ylabel='latitude',
        margins={'top': 10, 'right': 20, 'bottom': 20, 'left': 50},
        xticks=[35,45,55],
        ))

c.add_subplot(8,2,8, margin=0.01).add_child(
    crossfolium.BarFilter(
        c,
        'lng', width=400, height=100,
        bar_padding=0.1, domain=[-10,30],
        groupby=2,
        elastic_y=True,
        xlabel='',
        ylabel='longitude',
        margins={'top': 10, 'right': 20, 'bottom': 20, 'left': 50},
        xticks=[-10,0,10,20,30],
        ))

c.add_subplot(2,1,2, margin=0.01).add_child(
    crossfolium.TableFilter(c, data.columns.values.tolist()
    ))

f